### CSV

#### Saving SQLite database tables to CSV


In [ ]:
import sqlite3
import pandas as pd

# Connect to SQLite database
database_path = "../Datasets/database/mfa.db"
conn = sqlite3.connect(database_path)

# Get all table names
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

# Loop through each table and save it as a CSV
for table_name in tables["name"]:
    df = pd.read_sql(f"SELECT * FROM {table_name};", conn)
    df.to_csv(f"{table_name}.csv", index=False)
    # print(f"Saved {table_name}.csv")

# Close the connection
conn.close()

Saved collections.csv
Saved artists.csv
Saved created.csv


#### Creating a SQLite database from CSV files


The process involves:  

1. **Defining the Schema** – Deleting existing tables (if they exist) and creating new ones with appropriate constraints.  
2. **Loading CSV Data** – Reading data from CSV files into Pandas DataFrames.  
3. **Inserting Data into the Database** – Writing the CSV data into the corresponding SQLite tables.  

In [ ]:
import sqlite3
import pandas as pd
import os

# Define CSV file paths
DATA_DIR = '../Datasets/csv/mfa/'
ARTISTS_CSV = os.path.join(DATA_DIR, 'artists.csv')
COLLECTIONS_CSV = os.path.join(DATA_DIR, 'collections.csv')
CREATED_CSV = os.path.join(DATA_DIR, 'created.csv')

# Define and create database file path
DB_PATH = "Outputs/art_museum.db"

# Connect to SQLite database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

try:
    # Step 1: Delete existing tables if they exist
    cursor.executescript("""
        DROP TABLE IF EXISTS collections;
        DROP TABLE IF EXISTS artists;
        DROP TABLE IF EXISTS created;
    """)
    
    # Step 2: Create tables
    cursor.executescript("""
        CREATE TABLE collections (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            accession_number TEXT NOT NULL UNIQUE,
            acquired NUMERIC
        );
        
        CREATE TABLE artists (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL
        );
        
        CREATE TABLE created (
            artist_id INTEGER,
            collection_id INTEGER,
            PRIMARY KEY (artist_id, collection_id),
            FOREIGN KEY (artist_id) REFERENCES artists(id),
            FOREIGN KEY (collection_id) REFERENCES collections(id)
        );
    """)
    
    conn.commit()
    
    # Step 3: Load CSVs into DataFrames
    artists_df = pd.read_csv(ARTISTS_CSV)
    collections_df = pd.read_csv(COLLECTIONS_CSV)
    created_df = pd.read_csv(CREATED_CSV)
    
    # Step 4: Import CSV data into SQLite
    artists_df.to_sql("artists", conn, if_exists='append', index=False)
    collections_df.to_sql("collections", conn, if_exists='append', index=False)
    created_df.to_sql("created", conn, if_exists='append', index=False)
    
    conn.commit()
    print(f"Database created and saved successfully at {DB_PATH}.")

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()


Database created and saved successfully at Examples/art_museum.db.


In [ ]:
# Load and test:

import sqlite3

# Define database path
DB_PATH = "Outputs/art_museum.db"

# Connect to the database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Test: Check if tables exist
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables in database:", tables)

# Test: Retrieve a few records from each table
for table in ["artists", "collections", "created"]:
    print(f"\nTesting table: {table}")
    cursor.execute(f"SELECT * FROM {table} LIMIT 5;")
    rows = cursor.fetchall()
    for row in rows:
        print(row)

# Close connection
conn.close()


Tables in database: [('collections',), ('artists',), ('created',)]

Testing table: artists
(1, 'Li Yin')
(2, 'Qian Weicheng')
(3, 'Unidentified artist')
(4, 'Zhou Chen')

Testing table: collections
(1, 'Farmers working at dawn', '11.6152', '1911-08-03')
(2, 'Imaginative landscape', '56.496', None)
(3, 'Profusion of flowers', '56.257', '1956-04-12')
(4, 'Spring outing', '14.76', '1914-01-08')

Testing table: created
(1, 2)
(2, 3)
(3, 1)
(4, 4)


#### Load data from an xlsx spreadsheet to a database.

In [ ]:
import sqlite3
import pandas as pd
import os

# Define file paths
EXCEL_FILE = "../Datasets/excel/mfa.xlsx"
DB_PATH = "Outputs/art_museum_excel.db"

# Connect to SQLite database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

try:
    # Step 1: Delete existing tables if they exist
    cursor.executescript("""
        DROP TABLE IF EXISTS collections;
        DROP TABLE IF EXISTS artists;
        DROP TABLE IF EXISTS created;
    """)
    
    # Step 2: Create tables
    cursor.executescript("""
        CREATE TABLE collections (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            accession_number TEXT NOT NULL UNIQUE,
            acquired NUMERIC
        );
        
        CREATE TABLE artists (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL
        );
        
        CREATE TABLE created (
            artist_id INTEGER,
            collection_id INTEGER,
            PRIMARY KEY (artist_id, collection_id),
            FOREIGN KEY (artist_id) REFERENCES artists(id),
            FOREIGN KEY (collection_id) REFERENCES collections(id)
        );
    """)
    
    conn.commit()

    # Step 3: Load Excel sheets into DataFrames
    df_dict = pd.read_excel(EXCEL_FILE, sheet_name=None)  # Load all sheets

    # Extract DataFrames
    artists_df = df_dict.get("artists", pd.DataFrame())  # Ensure it exists
    collections_df = df_dict.get("collections", pd.DataFrame())
    created_df = df_dict.get("created", pd.DataFrame())

    # Step 4: Import data into SQLite
    if not artists_df.empty:
        artists_df.to_sql("artists", conn, if_exists='append', index=False)
    if not collections_df.empty:
        collections_df.to_sql("collections", conn, if_exists='append', index=False)
    if not created_df.empty:
        created_df.to_sql("created", conn, if_exists='append', index=False)

    conn.commit()
    print(f"Database created and saved successfully at {DB_PATH}.")

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    conn.close()


Database created and saved successfully at Examples/art_museum_excel.db.


In [ ]:
# put schema in a text file #todo
